In [ ]:

!pip install pillow_heif

import cv2,numpy as np,matplotlib.pyplot as plt
from google.colab import files
from IPython.display import Image,display
import pillow_heif
from PIL import Image as PILImage
import os

uploaded=files.upload()
filename=next(iter(uploaded))

if filename.lower().endswith('.heic'):
    heif_file=pillow_heif.read_heif(filename)
    image=PILImage.frombytes(heif_file.mode,heif_file.size,heif_file.data,"raw",heif_file.mode,heif_file.stride)
    image.save('temp.jpg','JPEG')
    image=cv2.imread('temp.jpg')
    os.remove('temp.jpg')
else:
    image=cv2.imread(filename)

image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.imshow(image_rgb)
plt.title('Input')
plt.axis('off')


In [ ]:
params={'hsv_lower':np.array([0,0,200]),'hsv_upper':np.array([180,30,255]),'min_area':100,'max_area':50000}

def detect(image,params):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,params['hsv_lower'],params['hsv_upper'])
    
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    
    contours,_=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    result=image.copy()
    objects=[]
    
    # Find all white regions
    white_regions=[]
    for c in contours:
        area=cv2.contourArea(c)
        if area>params['min_area']:
            M=cv2.moments(c)
            if M['m00']!=0:
                cx=int(M['m10']/M['m00'])
                cy=int(M['m01']/M['m00'])
                white_regions.append((cx,cy,area))
    
    # If we have multiple white regions, check if they form a circle
    if len(white_regions)>=3:
        # Find center point of all regions
        avg_x=sum(x for x,y,_ in white_regions)/len(white_regions)
        avg_y=sum(y for x,y,_ in white_regions)/len(white_regions)
        center=(int(avg_x),int(avg_y))
        
        # Calculate distances from center to each region
        distances=[np.sqrt((x-avg_x)**2+(y-avg_y)**2) for x,y,_ in white_regions]
        avg_radius=sum(distances)/len(distances)
        
        # Check if points form a roughly circular pattern
        is_circular=True
        for d in distances:
            if abs(d-avg_radius)/avg_radius>0.3:  # Allow 30% deviation
                is_circular=False
                break
        
        if is_circular:
            radius=int(avg_radius*1.2)  # Slightly larger circle to encompass all holes
            cv2.circle(result,center,radius,(0,255,0),3)
            cv2.putText(result,"Ball",center,cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),3)
            objects.append((center[0],center[1],"Ball"))
    
    return result,mask,objects

result,mask,objects=detect(image,params)
result_rgb=cv2.cvtColor(result,cv2.COLOR_BGR2RGB)

plt.subplot(122)
plt.imshow(result_rgb)
plt.title(f'Objects: {len(objects)}')
plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,5))
plt.imshow(mask,cmap='gray')
plt.title('Mask')
plt.axis('off')
plt.show()

print("\nDetected Objects:")
for i,(x,y,label) in enumerate(objects,1):
    print(f"{i}. {label}")


# Need to adjust the detection? Here's how! 🔧

If the detector isn't finding the white balls correctly, you can adjust these settings in the `white_detector` dictionary:

## If the wrong color is being detected:
Change these numbers in `white_detector`:
```python
'hsv_lower': np.array([0, 0, 200])   
'hsv_upper': np.array([180, 30, 255]) 
```

## If wrong sized objects are being detected:
Change these numbers:
```python
'min_area': 100 
'max_area': 5000 
```

Just change the numbers and run the cell again to see what happens! 🔄
Try small changes first - for example, change 200 to 180 or 220.


# Object Detection with OpenCV
This notebook demonstrates:
1. AprilTag detection using the `apriltag` library
2. White ball detection using HSV color thresholding
3. Visualization of detected objects

First, let's install the required packages:
